In [292]:
import pandas as pd
import os
import json
from pyecharts.charts import Graph
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import re
import difflib
from pyecharts.commons.utils import JsCode
from bs4 import BeautifulSoup as bs
import base64
from ctypes import c_uint
import binascii

In [82]:
quest_map = {}
quest_line_map = {}
quest_id_set = set()
line_id_set = set()
quest_zh_map = {}
quest_zh_line_map = {}
with open("lang/en_US.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "gtnh.quest" in line and "=" in line:
            one_line = line.split("=")
            quest_id = one_line[0].split(".")[1]
            if "name" in one_line[0]:
                quest_map[quest_id + "name"] = one_line[1]
            elif "desc" in one_line[0]:
                quest_map[quest_id + "desc"] = one_line[1]
            quest_id_set.add(quest_id)
        elif "gtnh.line" in line and "=" in line:
            one_line = line.split("=")
            quest_line_id = one_line[0].split(".")[1]
            if "name" in one_line[0]:
                quest_line_map[quest_line_id + "name"] = one_line[1]
            elif "desc" in one_line[0]:
                quest_line_map[quest_line_id + "desc"] = one_line[1]
            line_id_set.add(quest_line_id)

with open("lang/zh_CN.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "gtnh.quest" in line and "=" in line:
            one_line = line.split("=")
            quest_id = one_line[0].split(".")[1]
            if "name" in one_line[0]:
                quest_zh_map[quest_id + "name"] = one_line[1]
            elif "desc" in one_line[0]:
                quest_zh_map[quest_id + "desc"] = one_line[1]
            quest_id_set.add(quest_id)
        elif "gtnh.line" in line and "=" in line:
            one_line = line.split("=")
            quest_line_id = one_line[0].split(".")[1]
            if "name" in one_line[0]:
                quest_zh_line_map[quest_line_id + "name"] = one_line[1]
            elif "desc" in one_line[0]:
                quest_zh_line_map[quest_line_id + "desc"] = one_line[1]
            line_id_set.add(quest_line_id)

quest_name = pd.DataFrame(columns=["id", "name", "desc", "zh_name", "zh_desc"])
for i in quest_id_set:
    name = quest_map[i + "name"].strip() if (i + "name") in quest_map else ""
    desc = quest_map[i + "desc"].strip() if (i + "desc") in quest_map else ""
    zh_name = quest_zh_map[i + "name"].strip() if (i +
                                                   "name") in quest_zh_map else ""
    zh_desc = quest_zh_map[i + "desc"].strip()if (i +
                                                  "desc") in quest_zh_map else ""
    quest_name.loc[len(quest_name)] = [i.replace(
        "quest", ""), name, desc, zh_name, zh_desc]

quest_line_name = pd.DataFrame(
    columns=["id", "name", "desc", "zh_name", "zh_desc"])
for i in line_id_set:
    name = quest_line_map[i + "name"].strip() if (i +
                                                  "name") in quest_line_map else ""
    desc = quest_line_map[i + "desc"].strip() if (i +
                                                  "desc") in quest_line_map else ""
    zh_name = quest_zh_line_map[i +
                                "name"].strip() if (i + "name") in quest_zh_line_map else ""
    zh_desc = quest_zh_line_map[i +
                                "desc"].strip()if (i + "desc") in quest_zh_line_map else ""
    quest_line_name.loc[len(quest_line_name)] = [i.replace(
        "line", ""), name, desc, zh_name, zh_desc]

In [83]:
df = pd.DataFrame(columns=["quest_file_name", "quest_id",
                  "quest_name", "quest_desc", "quest_icon", "pre_quests"])
for quest_line in os.listdir("DefaultQuests/Quests/"):
    for quest in os.listdir(f"DefaultQuests/Quests/{quest_line}/"):
        que = json.load(
            open(f"DefaultQuests/Quests/{quest_line}/{quest}", "r", encoding="utf-8"))
        
        quest_id = que["questIDLow:4"]
        quest_name = que["properties:10"]["betterquesting:10"]["name:8"]
        quest_desc = que["properties:10"]["betterquesting:10"]["desc:8"]
        icon_name = "quests_icons/QuestIcon/" + str(quest_id) + ".png"
        
        pre_quests = []
        for i in que["preRequisites:9"]:
            pre_quests.append(que["preRequisites:9"][i]["questIDLow:4"])

        if f"quest{quest_id}name" in quest_zh_map:
            quest_name = quest_zh_map[f"quest{quest_id}name"]
        if f"quest{quest_id}desc" in quest_zh_map:
            quest_desc = quest_zh_map[f"quest{quest_id}desc"]

        quest_name = re.sub("(§.)", "", quest_name.strip())
        quest_desc = re.sub("(§.)", "", quest_desc.strip())

        df.loc[len(df)] = [quest, quest_id, quest_name,quest_desc, icon_name, pre_quests]


repeats = list(set(df[df["quest_name"].duplicated()]["quest_name"]))
for word in repeats:
    counter = 1
    for i in df[df["quest_name"] == word].sort_values("quest_id").index:
        df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
        counter += 1

In [322]:
for questline in os.listdir("DefaultQuests/QuestLines"):
    nodes = []
    links = []
    for quest in os.listdir(f"DefaultQuests/QuestLines/{questline}"):
        multi = 1
        if quest == "QuestLine.json":
            continue
        elif "Tier5IV" in quest:
            multi = 0.5
        que = json.load(
            open(f"DefaultQuests/QuestLines/{questline}/{quest}", "r", encoding="utf-8"))
        line = df[df["quest_id"] == que["questIDLow:4"]]

        desc = line["quest_desc"].values[0]

        temp_list = []
        for i in range(0, len(desc), 50):
            temp_list.append(desc[i : i + 50])

        path = f"image://version/272/{line["quest_icon"].values[0]}"

        name = line["quest_name"].values[0]

        if "questIDHigh:4" in que:
            high_id = que["questIDHigh:4"]
        else:
            high_id = 0
        low_id = que["questIDLow:4"]

        quest_id_str = base64.urlsafe_b64encode(bytes.fromhex((high_id.to_bytes(length=8, signed=True) + low_id.to_bytes(length=8, signed=True)).hex())).decode("utf-8")

        new_node = {"name": name,
                    "symbolSize": int(que["sizeX:3"]) * 1.3 * multi,
                        "symbol": path,
                        "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
                        "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
                        "data": desc,
                        "quest_id": quest_id_str,
                        }

        nodes.append(new_node)
        pre_quest_names = []
        pre_quests = line["pre_quests"].values[0]
        for quest_id in pre_quests:
            pre_que = df[df["quest_id"] == quest_id]
            pre_name = pre_que["quest_name"].values[0]
            pre_quest_names.append(pre_name)
            links.append({"source": pre_name,
                          "target": name})
            
        new_node["tooltip"] = "<strong>" + name + \
            "</strong></br>前置任务需求:</br>" + "</br>".join(pre_quest_names)

    g = (
        Graph(init_opts=opts.InitOpts(width="100%",
              height="900%", bg_color="#f5f0d3", chart_id="this_chart"))
        .add("", nodes, links, is_draggable=False, edge_symbol=['circle', 'arrow'], repulsion=0, label_opts=opts.LabelOpts(position="up", is_show=False), gravity=0, layout="none", edge_symbol_size=10)
        .render(f"html/{questline.split("-")[0]}.html")
    )

for html in os.listdir("html"):
    if not "html" in html:
        continue
    with open(f"html/{html}", encoding="utf-8") as page:
        a = bs(page)
    os.remove(f"html/{html}")
    a = a.find_all("script")[1]
    with open(f"js/{html.replace(".html", ".js")}", "w", encoding="utf-8") as js:
        js.write(a.text)

In [316]:
quest

'ZoomingwithNitro-AAAAAAAAAAAAAAAAAAALyA==.json'

In [61]:
quest_line_order = []
for line in open("DefaultQuests/QuestLinesOrder.txt").readlines():
    quest_line_order.append(line.split(": ")[1].strip())

In [6]:
counter = 0
with open("lang/en_US.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "=" in line:
            counter += 1
counter

6626

In [7]:
counter = 0
with open("lang/zh_CN.lang", "r", encoding="utf-8") as f:
    for line in f.readlines():
        if "=" in line:
            counter += 1
counter

6684

In [90]:
int(base64.b64decode("LIZCdBLfQquQpULGmFDixw==".encode("ascii")).hex(), 16)

59183147428921293450463928407402013383

In [79]:
base64.b64encode(bytes.fromhex("0" * (32-3) + hex(260)[2:])).decode("utf-8")

'AAAAAAAAAAAAAAAAAAABBA=='

In [314]:
high_id = hex(que["questIDHigh:4"]).replace(
    "0x", "").replace("-", "").rjust(16, "0")

low_id = hex(que["questIDLow:4"]).replace(
    "0x", "").replace("-", "").rjust(16, "0")
base64.b64encode(bytes.fromhex((que["questIDHigh:4"].to_bytes(length=8, signed=True) +
                                que["questIDLow:4"].to_bytes(length=8, signed=True)).hex())).decode("utf-8")

'LIZCdBLfQquQpULGmFDixw=='

In [238]:
"LIZCdBLfQquQpULGmFDixw=="

'LIZCdBLfQquQpULGmFDixw=='

In [151]:
hex(int(bin(que["questIDLow:4"] & 0xffffffff), 2))

'0x9850e2c7'

In [167]:
bytes.fromhex((bin(que["questIDHigh:4"]).replace("0b", "").replace(
    "-0b", "").rjust(64, "0") + bin(que["questIDLow:4"]).replace(
    "-0b", "").rjust(64, "0")))

b'\x00\x10\x11\x00\x10\x00\x01\x10\x01\x00\x00\x10\x01\x11\x01\x00\x00\x01\x00\x10\x11\x01\x11\x11\x01\x00\x00\x10\x10\x10\x10\x11\x01\x10\x11\x11\x01\x01\x10\x10\x10\x11\x11\x01\x00\x11\x10\x01\x01\x10\x01\x11\x10\x10\x11\x11\x00\x01\x11\x01\x00\x11\x10\x01'

In [159]:
len(bin(que["questIDHigh:4"]).replace("0b", "").replace(
    "-0b", "").rjust(64, "0") + bin(que["questIDLow:4"]).replace(
    "-0b", "").rjust(64, "0"))

128

In [245]:
hex(int(bin(que["questIDHigh:4"])[2:].rjust(
    64, "0") + bin(c_uint(que["questIDLow:4"]).value)[2:].rjust(64, "0"), 2))[2:].rjust(32, "0")

'2c86427412df42ab000000009850e2c7'

In [234]:
~(que["questIDLow:4"] - 1)

8023933740364275001

In [280]:
que["questIDHigh:4"].to_bytes(length=16, signed=True)+que["questIDLow:4"].to_bytes(length=16, signed=True)

b'\x00\x00\x00\x00\x00\x00\x00\x00,\x86Bt\x12\xdfB\xab\xff\xff\xff\xff\xff\xff\xff\xff\x90\xa5B\xc6\x98P\xe2\xc7'

In [311]:
(que["questIDHigh:4"].to_bytes(length=8, signed=True) + \
    que["questIDLow:4"].to_bytes(length=8, signed=True)).hex()

'2c86427412df42ab90a542c69850e2c7'